split1

In [1]:
split = 'split1'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

print(train_path)
print(test_path)
print(val_path)

../5-fold_new/DATA//split1/train/train_split1.csv
../5-fold_new/DATA//test.csv
../5-fold_new/DATA//split1/val/val_split1.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)
print('\n')

print("Shape of training set:", x_train.shape)
print("Shape of testing set:", x_test.shape)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


Shape of training set: (364, 30)
Shape of testing set: (114, 30)


In [3]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

svc_model = SVC(kernel="rbf")
svc_model.fit(x_train, y_train)
predictions = svc_model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

svm_acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM model is: ", svm_acc)

Confusion Matrix: 
 [[46  0]
 [24 44]]


              precision    recall  f1-score   support

           0       0.66      1.00      0.79        46
           1       1.00      0.65      0.79        68

    accuracy                           0.79       114
   macro avg       0.83      0.82      0.79       114
weighted avg       0.86      0.79      0.79       114

Accuracy of SVM model is:  0.7894736842105263


split2

In [4]:
split = 'split2'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [5]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

svc_model = SVC(kernel="rbf")
svc_model.fit(x_train, y_train)
predictions = svc_model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

svm_acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM model is: ", svm_acc)

Confusion Matrix: 
 [[46  0]
 [23 45]]


              precision    recall  f1-score   support

           0       0.67      1.00      0.80        46
           1       1.00      0.66      0.80        68

    accuracy                           0.80       114
   macro avg       0.83      0.83      0.80       114
weighted avg       0.87      0.80      0.80       114

Accuracy of SVM model is:  0.7982456140350878


split3

In [6]:
split = 'split3'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [7]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

svc_model = SVC(kernel="rbf")
svc_model.fit(x_train, y_train)
predictions = svc_model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

svm_acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM model is: ", svm_acc)

Confusion Matrix: 
 [[45  1]
 [20 48]]


              precision    recall  f1-score   support

           0       0.69      0.98      0.81        46
           1       0.98      0.71      0.82        68

    accuracy                           0.82       114
   macro avg       0.84      0.84      0.82       114
weighted avg       0.86      0.82      0.82       114

Accuracy of SVM model is:  0.8157894736842105


split4

In [8]:
split = 'split4'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [9]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

svc_model = SVC(kernel="rbf")
svc_model.fit(x_train, y_train)
predictions = svc_model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

svm_acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM model is: ", svm_acc)

Confusion Matrix: 
 [[46  0]
 [20 48]]


              precision    recall  f1-score   support

           0       0.70      1.00      0.82        46
           1       1.00      0.71      0.83        68

    accuracy                           0.82       114
   macro avg       0.85      0.85      0.82       114
weighted avg       0.88      0.82      0.83       114

Accuracy of SVM model is:  0.8245614035087719


split5

In [10]:
split = 'split5'
path = "../5-fold_new/DATA/"
train_path = "{}/{}/train/train_{}.csv".format(path,split,split)
test_path =  "{}/test.csv".format(path)
val_path = "{}/{}/val/val_{}.csv".format(path,split,split)

# Load the training set
train_data = pd.read_csv(train_path)
print(train_data.shape)
x_train = train_data[train_data.columns[2:]]
print(x_train.shape)
y_train = train_data[train_data.columns[1]]
le = LabelEncoder()
y_train = np.array(le.fit_transform(y_train))
print(le.classes_)
print('\n')

# Load the testing set
test_data = pd.read_csv(test_path)
print(test_data.shape)
x_test = test_data[test_data.columns[3:]]
print(x_test.shape)
y_test = test_data[test_data.columns[2]]
le = LabelEncoder()
y_test = np.array(le.fit_transform(y_test))
print(le.classes_)
print('\n')

# Load the validation set
val_data = pd.read_csv(val_path)
print(val_data.shape)
x_val = val_data[val_data.columns[2:]]
print(x_val.shape)
y_val = val_data[val_data.columns[1]]
le = LabelEncoder()
y_val = np.array(le.fit_transform(y_val))
print(le.classes_)

(364, 32)
(364, 30)
['B' 'M']


(114, 33)
(114, 30)
['B' 'M']


(91, 32)
(91, 30)
['B' 'M']


In [11]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

svc_model = SVC(kernel="rbf")
svc_model.fit(x_train, y_train)
predictions = svc_model.predict(x_test)

print("Confusion Matrix: \n", confusion_matrix(y_test, predictions))
print("\n")
print(classification_report(y_test, predictions))

svm_acc = accuracy_score(y_test, predictions)
print("Accuracy of SVM model is: ", svm_acc)

Confusion Matrix: 
 [[46  0]
 [17 51]]


              precision    recall  f1-score   support

           0       0.73      1.00      0.84        46
           1       1.00      0.75      0.86        68

    accuracy                           0.85       114
   macro avg       0.87      0.88      0.85       114
weighted avg       0.89      0.85      0.85       114

Accuracy of SVM model is:  0.8508771929824561
